In [ ]:
# import libraries
import arcpy
import os
import ee
import geemap

In [ ]:
# Create Map
Map = geemap.Map()
Map

In [ ]:
# create roi
roi = ee.FeatureCollection(Map.draw_last_feature)

In [ ]:
# Create median composite
s2_collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterBounds(roi)
    .filterDate('2024-01-01', '2024-07-31')
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 10))
)

image = s2_collection.median().clip(roi)

In [ ]:
# True Color (Natural Color)
vis_params_truecolor = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 3000,
    'gamma': 1.2
}

# Add layers to map
Map.addLayer(image, vis_params_truecolor, 'True Color')
Map.centerObject(image)
Map

In [ ]:
# Export Image
out_dir = os.path.join(os.path.expanduser("~"), "Downloads")
filename = os.path.join(out_dir, "sentinel2_image.tif")
geemap.ee_export_image(
    image.select(['B1', 'B2', 'B3', 'B4', 'B6', 'B7','B8',]), 
    filename=filename, 
    scale=10, 
    region=roi.geometry(), 
    file_per_band=False
)

In [ ]:
# Add exported image to map
aprx = arcpy.mp.ArcGISProject("CURRENT")
aprx.activeMap.addDataFromPath(filename)